# NETWORK INTRUSION DETECTION SYSTEM (NIDS)
## Exploratory Data Analysis - CIC-IDS-2017

---

**Autore:** Emanuele Pascale
**Anno Accademico:** 2025/2026  
**Data:** Febbraio 2026

---

### Riferimenti Scientifici

> **Sharafaldin, I., Lashkari, A. H., & Ghorbani, A. A.** (2018). *Toward Generating a New Intrusion Detection Dataset and Intrusion Traffic Characterization*. 4th International Conference on Information Systems Security and Privacy (ICISSP), 108-116.

**Dataset Source:** [Canadian Institute for Cybersecurity - UNB](https://www.unb.ca/cic/datasets/ids-2017.html)

---

### Obiettivi dell'Analisi

Questa **Exploratory Data Analysis (EDA)** ha l'obiettivo di:

1. **Comprendere la struttura** del dataset CIC-IDS-2017 e identificare eventuali problematiche di qualità
2. **Analizzare la distribuzione** delle classi di attacco e valutare il class imbalance
3. **Preparare il terreno** per la fase di feature selection (Notebook 2)


---

### Struttura del Notebook

```
FASE 1-3: DATA LOADING & PREPROCESSING
├─ Sezione 1: Setup ambiente e configurazione
├─ Sezione 2: Definizione helper functions
├─ Sezione 3: Caricamento dataset (8 file CSV)
├─ Sezione 4: Data cleaning (NaN, Inf, duplicati, negativi)
└─ Sezione 5: Label engineering (15 → 8 categorie)

FASE 4-8: EXPLORATORY DATA ANALYSIS
├─ Sezione 6: Analisi distribuzione classi
├─ Sezione 7: Dataset overview e statistiche base
├─ Sezione 8: Zero/near-zero variance detection
├─ Sezione 9: Analisi univariata (distribuzioni)
├─ Sezione 10: Analisi bivariata (feature-target correlation)
├─ Sezione 11: Class separability visualization
├─ Sezione 12: Multicollinearity detection
└─ Sezione 13: Feature selection preparation (MI + ANOVA)

FASE 9: DOMAIN-SPECIFIC ANALYSIS
└─ Sezione 14: Analisi feature di rete (TCP flags, protocolli)

```

---

## SEZIONE 1: Setup Ambiente e Configurazione

### Obiettivo

In questa sezione iniziale configuriamo l'**ambiente di lavoro** importando le librerie necessarie e definendo i parametri globali per l'analisi.

### Note Implementative

- Impostiamo `random_seed=42` per **riproducibilità** dei risultati
- Configuriamo `warnings.filterwarnings('ignore')` per output pulito
- Utilizziamo `os.makedirs(exist_ok=True)` per creare directory se mancanti

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import glob
import gc
import warnings
import pickle
from datetime import datetime
from collections import Counter

# Scikit-learn: preprocessing e feature selection
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.feature_selection import (
    mutual_info_classif, 
    f_classif, 
    VarianceThreshold
)
from sklearn.decomposition import PCA

# Scipy: statistiche e correlazioni
from scipy import stats
from scipy.stats import spearmanr, pearsonr, mannwhitneyu

# Configurazione warnings (per output pulito)
warnings.filterwarnings('ignore')

print("Librerie importate con successo")


# CONFIGURAZIONE GLOBALE

# Seed per riproducibilità
np.random.seed(42)

# Stile grafici
sns.set_style("whitegrid")
sns.set_palette("husl")

# Parametri matplotlib
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 11

# Opzioni pandas display
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Configurazione globale applicata")



# CONFIGURAZIONE PATH

DATA_PATH = "../data"                      # Cartella con i file CSV del dataset
OUTPUT_PATH = "../output"                  # Cartella principale per output
IMG_PATH = os.path.join(OUTPUT_PATH, "eda_images")         # Grafici e visualizzazioni
FEATURES_PATH = os.path.join(OUTPUT_PATH, "feature_analysis")  # Analisi feature
REPORTS_PATH = os.path.join(OUTPUT_PATH, "reports")        # Report CSV/pickle

# Crea directory se non esistono
for path in [OUTPUT_PATH, IMG_PATH, FEATURES_PATH, REPORTS_PATH]:
    os.makedirs(path, exist_ok=True)

# Banner iniziale
print("=" * 80)
print(" " * 20 + "NIDS - CIC-IDS-2017 EDA DEFINITIVA")
print("=" * 80)
print(f" Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f" Data Path: {DATA_PATH}")
print(f" Output Path: {OUTPUT_PATH}")
print(f" Python: {sys.version.split()[0]} | Pandas: {pd.__version__} | NumPy: {np.__version__}")
print("=" * 80)
print()
print("Path configurati e directory create")

## Configurazione

**Seed casuale:** Impostare `np.random.seed(42)` garantisce che operazioni stocastiche (sampling, shuffle) producano sempre gli stessi risultati, quindi fondamentale per la **riproducibilità**.

**Struttura directory:**
```
output/
├── eda_images/        ← Grafici PNG ad alta risoluzione (300 dpi)
├── feature_analysis/  ← CSV con ranking feature
└── reports/           ← Report numerici e pickle objects
```

## SEZIONE 2: Helper Functions

### Obiettivo

Definire **funzioni di utilità** riutilizzabili per:
- Ottimizzazione memoria (conversione dtype)
- Monitoraggio uso RAM
- Visualizzazioni standardizzate
- Salvataggio report automatico

### Funzioni Implementate

| Funzione | Scopo | Input | Output |
|----------|-------|-------|--------|
| `optimize_dtypes()` | Riduce memoria 50% | DataFrame | DataFrame ottimizzato |
| `print_memory_usage()` | Mostra RAM usata | DataFrame | Print statement |
| `plot_distribution_comparison()` | Visualizza feature per classe | DataFrame, feature | Plot salvato |
| `save_report()` | Salva dati su file | DataFrame/dict, filename | File CSV/PKL |


In [ ]:
def optimize_dtypes(df):
    """
    Ottimizza l'uso della memoria convertendo dtype da 64bit a 32bit/16bit.
    
    Parametri:
        df (pd.DataFrame): DataFrame da ottimizzare
        
    Returns:
        pd.DataFrame: DataFrame con dtype ottimizzati
        
    Note:
        - float64 → float32 (riduzione 50% memoria)
        - int64 → int32 o int16 (basato su range valori)
        - Preserva categorie e stringhe
    """
    for col in df.columns:
        col_type = df[col].dtype
        if col_type == 'float64':
            df[col] = df[col].astype('float32')
        elif col_type == 'int64':
            # Scegli int16 o int32 in base al range
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype('int16')
            else:
                df[col] = df[col].astype('int32')
                
    return df

print("optimize_dtypes() definita")


def print_memory_usage(df, label=""):
    """
    Stampa informazioni sull'uso della memoria del DataFrame.

    Parametri:
        df (pd.DataFrame): DataFrame da analizzare
        label (str): Etichetta descrittiva per output

    Output:
        Print statement con memoria in MB e dimensioni DataFrame
    """
    memory_mb = df.memory_usage(deep=True).sum() / (1024 ** 2)
    print(f" {label}: {memory_mb:.2f} MB "
          f"({df.shape[0]:,} rows × {df.shape[1]} cols)")

print("print_memory_usage() definita")


def plot_distribution_comparison(df, feature, target='Label',
                                   save_path=None, log_scale=False):
    """
    Visualizza la distribuzione di una feature per ogni classe target.
    Utilizzata per analisi bivariate.

    Parametri:
        df (pd.DataFrame): Dataset completo
        feature (str): Nome della feature da visualizzare
        target (str): Nome della colonna target (default: 'Label')
        save_path (str): Path completo cocnesnelele per salvare figura (opzionale)
        log_scale (bool): Applica scala logaritmica all'asse y

    Output:
        Figura con 2 subplot (violin plot + box plot)
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Subplot 1: Violin plot (mostra distribuzione completa)
    sns.violinplot(x=target, y=feature, data=df, ax=axes[0],
                   inner='quartile', scale='width')
    axes[0].set_title(f'Distribuzione: {feature} per Classe')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(True, alpha=0.3, axis='y')

    if log_scale:
        axes[0].set_yscale('log')

    # Subplot 2: Box plot (focus su outliers)
    sns.boxplot(x=target, y=feature, data=df, ax=axes[1],
                showfliers=False)  # No outliers per leggibilità
    axes[1].set_title(f'Box Plot: {feature} per Classe')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(True, alpha=0.3, axis='y')

    if log_scale:
        axes[1].set_yscale('log')

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"   Salvato: {os.path.basename(save_path)}")

    plt.show()
    plt.close()

print("plot_distribution_comparison() definita")

import json

def save_report(data, filename, report_type='csv'):
    """
    Salva report in formati LEGGIBILI (CSV o JSON).
    Gestisce automaticamente liste e dizionari.
    """
    filepath = os.path.join(REPORTS_PATH, filename)

    if report_type == 'csv':
        # Se è una lista semplice, la converte in DataFrame al volo
        if isinstance(data, list):
            pd.DataFrame(data, columns=['Value']).to_csv(filepath, index=False)
        elif isinstance(data, pd.DataFrame):
            data.to_csv(filepath, index=False)

    elif report_type == 'json':
        # Helper per convertire numeri NumPy (int64, float32) in tipi Python standard per JSON
        def convert(o):
            if isinstance(o, (np.int64, np.int32, np.int16)): return int(o)
            if isinstance(o, (np.float64, np.float32)): return float(o)
            if isinstance(o, np.ndarray): return o.tolist()
            return str(o)

        with open(filepath, 'w') as f:
            json.dump(data, f, indent=4, default=convert)

    print(f"Report salvato: {filename}")

print("save_report() definita")
print()
print("Tutte le helper functions sono state caricate con successo")

### Interpretazione delle Helper Functions

**Ottimizzazione memoria:** La funzione `optimize_dtypes()` è cruciale:
- Permette di liberare circa il 50% di memoria utilizzata dai float64 e int64
- Speedup operazioni grazie a minore I/O memoria

---

## SEZIONE 3: Data Loading

### Obiettivo

Caricare in memoria gli **8 file CSV** che compongono il dataset CIC-IDS-2017 e concatenarli in un unico DataFrame.

### Struttura Dataset Originale

Il dataset è distribuito in 8 file corrispondenti a diverse giornate di traffico:

| File | Giorno | Attacchi Presenti | Dimensione (~) |
|------|--------|-------------------|----------------|
| `Monday-WorkingHours.pcap_ISCX.csv` | Lunedì | Solo BENIGN | ~530k righe |
| `Tuesday-WorkingHours.pcap_ISCX.csv` | Martedì | FTP/SSH Brute Force | ~445k righe |
| `Wednesday-workingHours.pcap_ISCX.csv` | Mercoledì | DoS/Heartbleed | ~692k righe |
| `Thursday-Morning-WebAttacks.pcap_ISCX.csv` | Giovedì AM | Web Attacks | ~170k righe |
| `Thursday-Afternoon-Infilteration.pcap_ISCX.csv` | Giovedì PM | Infiltration | ~288k righe |
| `Friday-Morning.pcap_ISCX.csv` | Venerdì AM | Bot, PortScan (parziale) | ~191k righe |
| `Friday-Afternoon-DDos.pcap_ISCX.csv` | Venerdì PM | DDoS | ~225k righe |
| `Friday-Afternoon-PortScan.pcap_ISCX.csv` | Venerdì PM | PortScan | ~286k righe |

**Totale:** ~2.83M righe × 79 colonne

### Note Tecniche

1. **Encoding:** I file utilizzano encoding `cp1252` (Windows-1252), non UTF-8 standard
2. **Nomi colonne:** Contengono spazi extra (es. `" Flow Duration "`) che vanno rimossi con `.str.strip()`
3. **Memoria:** Caricamento file-per-file con ottimizzazione immediata previene RAM overflow
4. **Garbage collection:** `gc.collect()` dopo concatenazione libera memoria temporanea

---

In [ ]:
# STEP 1: Ricerca dei File

# Trova tutti i file .csv nella cartella DATA_PATH
all_files = sorted(glob.glob(os.path.join(DATA_PATH, "*.csv")))

if not all_files:
    raise FileNotFoundError(
        f"Nessun file CSV trovato in {DATA_PATH}.\n"
    )

print(f"Trovati {len(all_files)} file CSV:\n")
for i, filepath in enumerate(all_files, 1):
    print(f"   {i}. {os.path.basename(filepath)}")

print(f"\n   Caricamento in corso...\n")


# STEP 2: Caricamento File per File con Gestione Errori


dfs = []              # Lista per accumulare DataFrame
file_info = []        # Statistiche per report

for idx, filepath in enumerate(all_files, 1):
    basename = os.path.basename(filepath)
    try:
        # Carica CSV con encoding specifico CIC-IDS-2017
        df_temp = pd.read_csv(filepath, encoding='cp1252', low_memory=False)

        # Pulizia nomi colonne (rimuove spazi extra)
        df_temp.columns = df_temp.columns.str.strip()

        # Ottimizzazione memoria immediata
        df_temp = optimize_dtypes(df_temp)

        # Statistiche file
        file_info.append({
            'File': basename,
            'Rows': df_temp.shape[0],
            'Memory_MB': df_temp.memory_usage(deep=True).sum() / (1024**2)
        })

        dfs.append(df_temp)

        print(f"{basename:<55} | {df_temp.shape[0]:>9,} righe")

    except Exception as e:
        print(f"ERRORE su {basename}: {e}")

if not dfs:
    raise ValueError(" Nessun dataframe caricato correttamente.")

# STEP 3: Concatenazione e Pulizia Memoria

print(f"\nConcatenazione {len(dfs)} file in corso...")

# Concatena tutti i DataFrame con reset indici
df_raw = pd.concat(dfs, ignore_index=True)

# Libera memoria (lista temporanea non più necessaria)
del dfs
gc.collect()

print(f"\n Dataset concatenato con successo")
print_memory_usage(df_raw, "Dimensioni")

# Salva statistiche caricamento
file_stats_df = pd.DataFrame(file_info)
save_report(file_stats_df, '01_file_loading_stats.csv', 'csv')

**Prossimo step:** Data cleaning per rimuovere anomalie (NaN, duplicati, valori invalidi).

---

## SEZIONE 4: Data Cleaning

### Obiettivo

Identificare e rimuovere **anomalie nei dati** che comprometterebbero il training dei modelli:
1. Valori **infiniti** (risultato di divisioni per zero)
2. Valori **mancanti** (NaN)
3. Righe **duplicate**
4. Valori **negativi non validi** (es. durata negativa)

### Rationale Scientifico

> **Sharafaldin et al. (2018)** documentano nel paper originale che il tool **CICFlowMeter** (usato per generare il dataset) produce errori di calcolo in presenza di pacchetti malformati, risultando in valori infiniti o negativi per metriche temporali.

---

In [ ]:
initial_rows = df_raw.shape[0]
cleaning_log = {}  # Dizionario per tracciare operazioni


# STEP 1: Gestione Valori Infiniti

print("Step 1: Gestione valori Infiniti...")

# Sostituisco infiniti con NaN (che saranno rimossi nel prossimo step)
df_raw.replace([np.inf, -np.inf], np.nan, inplace=True)

print("   Valori infiniti convertiti in NaN")

# STEP 2: Rimozione Valori Mancanti (NaN)

print("\nStep 2: Rimozione valori mancanti (NaN)...")

rows_before_na = df_raw.shape[0]
df_raw.dropna(inplace=True)
nan_dropped = rows_before_na - df_raw.shape[0]
cleaning_log['NaN_removed'] = nan_dropped

print(f"   Rimossi {nan_dropped:,} record con NaN")


# STEP 3: Rimozione Duplicati

print("\nStep 3: Rimozione duplicati...")

rows_before_dedup = df_raw.shape[0]
df_raw.drop_duplicates(inplace=True)
dupes_dropped = rows_before_dedup - df_raw.shape[0]
cleaning_log['Duplicates_removed'] = dupes_dropped

print(f"   Rimossi {dupes_dropped:,} record duplicati")

# STEP 4: SANITY CHECK

print("\nStep 4: Sanity Check (Valori Negativi Non Validi)...")

# Feature che NON possono essere negative (fisica/temporale impossibile)
# NOTA: Escludiamo 'Init_Win_bytes_forward/backward' dove -1 = missing value legittimo
cols_must_be_positive = [
    'Flow Duration',      # Durata temporale (>= 0)
    'Flow Bytes/s',       # Rate trasmissione (>= 0)
    'Flow Packets/s',     # Rate pacchetti (>= 0)
    'Flow IAT Mean',      # Inter-arrival time medio (>= 0)
    'Fwd IAT Total',      # Tempo totale forward (>= 0)
    'Bwd IAT Total'       # Tempo totale backward (>= 0)
]

# Verifica esistenza colonne (gestione spazi nei nomi)
target_cols = [c for c in cols_must_be_positive if c in df_raw.columns]

if not target_cols:
    print("    Nessuna colonna target trovata per sanity check, skip.")
    cleaning_log['Negative_values_removed'] = 0
else:
    print(f"   Controllo valori negativi su {len(target_cols)} colonne:")

    # Conta righe con ALMENO UNA feature negativa
    neg_mask = (df_raw[target_cols] < 0).any(axis=1)
    neg_dropped = neg_mask.sum()

    if neg_dropped > 0:
        # Dettaglio per colonna (diagnostico)
        print("\n    Breakdown per colonna:")
        for col in target_cols:
            neg_count = (df_raw[col] < 0).sum()
            if neg_count > 0:
                print(f"      • {col:<25}: {neg_count:>8,} valori negativi")

        # Rimuovi righe invalide
        rows_before_neg = df_raw.shape[0]
        df_raw = df_raw[~neg_mask].copy()

        cleaning_log['Negative_values_removed'] = neg_dropped
        print(f"\n Rimossi {neg_dropped:,} record con valori negativi invalidi (errore CICFlowMeter)")
    else:
        print("Nessun valore negativo invalido trovato")
        cleaning_log['Negative_values_removed'] = 0

# STEP 5: Reset Index e Report Finale

# Reset index dopo rimozioni
df_raw.reset_index(drop=True, inplace=True)

# Report finale cleaning
print()
print(f"   Righe Iniziali:        {initial_rows:>12,}")
print(f"   - NaN/Inf rimossi:     {nan_dropped:>12,}")
print(f"   - Duplicati rimossi:   {dupes_dropped:>12,}")
print(f"   - Negativi rimossi:    {neg_dropped:>12,}")
print()
print(f"{'-' * 60}")
print(f"   RIGHE VALIDE:       {df_raw.shape[0]:>12,} "
      f"({100 * df_raw.shape[0] / initial_rows:.1f}% retained)")
print(f"{'-' * 60}\n")

print_memory_usage(df_raw, "Dataset Pulito")

# Salva log cleaning per tracciabilità
save_report(cleaning_log, '02_cleaning_log.json', 'json')


## SEZIONE 5: Label Engineering

### Obiettivo

Trasformare le **15 label originali** in **8 macro-categorie** semanticamente coerenti per:
1. Ridurre complessità problema di classificazione
2. Raggruppare attacchi con caratteristiche simili
3. Migliorare bilanciamento classi

### Tassonomia Originale

Il dataset CIC-IDS-2017 contiene le seguenti label:

```
BENIGN                         ← Traffico normale
DoS Hulk                       ← Denial of Service variant
DoS GoldenEye                  ← Denial of Service variant
DoS slowloris                  ← Denial of Service variant
DoS Slowhttptest               ← Denial of Service variant
DDoS                           ← Distributed DoS
FTP-Patator                    ← Brute force FTP
SSH-Patator                    ← Brute force SSH
PortScan                       ← Port scanning
Bot                            ← Botnet IRC
Web Attack – Brute Force       ← Web application attack
Web Attack – XSS               ← Cross-site scripting
Web Attack – Sql Injection     ← SQL injection
Infiltration                   ← Multi-step penetration
Heartbleed                     ← SSL/TLS vulnerability exploit
```

### Note Tecniche

- **Encoding UTF-8:** Alcune label hanno caratteri corrotti.
- **LabelEncoder:** Convertiamo encoding numerico per target integer.

---

In [ ]:
# STEP 1: Analisi Label Originali

print("  FASE 3: LABEL ENGINEERING")

print()

print(" Label originali uniche nel dataset:")
original_labels = df_raw['Label'].unique()
print(f"   Totale categorie: {len(original_labels)}\n")

for lbl in sorted(original_labels):
    count = (df_raw['Label'] == lbl).sum()
    print(f"   - {lbl:<40} : {count:>10,}")


# STEP 2: Definizione Mapping e Applicazione

print("\n Applicazione schema di raggruppamento...\n")

# Label Mapping secondo Sharafaldin et al. (2018)
label_mapping = {
    'BENIGN': 'Benign',

    # DoS Attacks (mono-source)
    'DoS Hulk': 'DoS',
    'DoS GoldenEye': 'DoS',
    'DoS slowloris': 'DoS',
    'DoS Slowhttptest': 'DoS',

    # DDoS (distributed)
    'DDoS': 'DDoS',

    # Brute Force
    'FTP-Patator': 'BruteForce',
    'SSH-Patator': 'BruteForce',

    # Port Scan
    'PortScan': 'PortScan',

    # Bot
    'Bot': 'Bot',

    # Web Attacks (gestione varianti encoding UTF-8)
    'Web Attack ï¿½ Brute Force': 'WebAttack',  # UTF-8 corrupted
    'Web Attack Brute Force': 'WebAttack',      # UTF-8 correct
    'Web Attack ï¿½ XSS': 'WebAttack',
    'Web Attack XSS': 'WebAttack',
    'Web Attack ï¿½ Sql Injection': 'WebAttack',
    'Web Attack Sql Injection': 'WebAttack',

    # Infiltration & Exploits
    'Infiltration': 'Other',
    'Heartbleed': 'Other'
}

# Applica mapping
df_raw['Label'] = df_raw['Label'].astype(str).str.strip().replace(label_mapping)

# Verifica risultato
final_labels = sorted(df_raw['Label'].unique())
print(f"  Label dopo raggruppamento: {len(final_labels)} categorie")
print(f"   {final_labels}\n")


# STEP 3: Encoding Numerico Label

print(" Creazione encoding numerico per label...\n")

# LabelEncoder per conversione string → int
le = LabelEncoder()
df_raw['Label_Encoded'] = le.fit_transform(df_raw['Label'])

# Mostra mapping
label_map_dict = dict(zip(le.classes_, le.transform(le.classes_)))
print("   Mapping Label → Codice Numerico:")
for label, code in sorted(label_map_dict.items(), key=lambda x: x[1]):
    print(f"      {code} : {label}")

# Salva encoder per utilizzo in Notebook 3
save_report(le.classes_.tolist(), '03_label_encoder_classes.json', 'json')


print("\n  Label engineering completato")
print(f"   Colonna 'Label': string categorica (8 classi)")
print(f"   Colonna 'Label_Encoded': integer encoding (0-7)\n")

### Interpretazione del Label Engineering

**Vantaggi del raggruppamento (15→8 classi):**
1. **Riduzione complessità:** Problema 8-class vs 15-class riduce spazio ipotesi modelli
2. **Miglior generalizzazione:** Attacchi simili (es. 4 varianti DoS) condividono pattern feature
3. **Bilanciamento parziale:** Le categorie rare (Heartbleed) vengono fuse con Infiltration

**Trade-off:**
- **Persa granularità:** Non distinguiamo più tra slowloris vs Hulk DoS
- **Mixing eterogeneo:** WebAttack include sia Brute Force web che XSS/SQL injection

**Prossimo step:** Analisi distribuzione classi per quantificare imbalance.

---

## SEZIONE 5.1: TCP Appendix Filtering

### Obiettivo

Rimuovere **artefatti TCP appendix** dal dataset, ovvero flussi di rete con un numero troppo basso di pacchetti che rappresentano connessioni incomplete o abortite, non campioni validi per l'addestramento.

### Problema Documentato

> **Engelen et al. (2021)** - *"Troubleshooting an Intrusion Detection Dataset: the CICIDS2017 Case Study"* documentano che il dataset CIC-IDS-2017 contiene una **significativa percentuale di TCP appendix artifacts**: flussi con 1-2 pacchetti totali che corrispondono a:
> 
> - **SYN scans** (singolo pacchetto SYN senza risposta)
> - **Connessioni TCP aborrite** (SYN + RST/ACK incompleti)
> - **Errori di estrazione CICFlowMeter** (timeout su connessioni non terminate)

Questi flussi **non rappresentano traffico di rete completo** e introducono noise nel training, degradando le performance dei modelli ML.

### Soluzione

Applicare un **threshold minimo di pacchetti** per flusso valido.
### Implementazione

Il filtro opera sulla **somma di pacchetti forward e backward**:

```python
total_packets = Total_Fwd_Packets + Total_Backward_Packets
valid_flows = (total_packets >= 3)
```


In [ ]:
## SEZIONE 5.1: TCP Appendix Filtering (Engelen et al. 2021)

print(" TCP Appendix Filtering (Engelen et al. 2021)")
print("-" * 80)

# Applica filtro: mantieni flussi con almeno 3 pacchetti totali
min_packet_threshold = 3

# Calcolo usando 'Total Fwd Packets' e 'Total Backward Packets'
total_packets = df_raw['Total Fwd Packets'] + df_raw['Total Backward Packets']

flows_before = len(df_raw)
df_raw = df_raw[total_packets >= min_packet_threshold].copy()
flows_after = len(df_raw)
flows_removed = flows_before - flows_after

print(f"Threshold impostato: {min_packet_threshold} pacchetti minimo per flusso")
print(f"   Flussi prima filtro:  {flows_before:,}")
print(f"   Flussi dopo filtro:   {flows_after:,}")
print(f"   Flussi rimossi:       {flows_removed:,} ({flows_removed/flows_before*100:.1f}%)")

if flows_removed > 0:
    print(f" Artefatti TCP appendix rimossi")
else:
    print(f" Nessun TCP appendix rilevato")


## SEZIONE 6: Analisi Distribuzione Classi

### Obiettivo

Analizzare la **distribuzione delle 8 categorie** di attacco per:
1. Quantificare il **class imbalance**
2. Identificare classi **maggioritarie** e **minoritarie**
3. Definire **strategie di handling** per il modeling

### Strategie di Mitigazione

Per class imbalance potremmo applicare:
1. **Resampling:** SMOTE (Synthetic Minority Oversampling), non adatto per questo dataset
2. **Algorithmic:** `class_weight='balanced'` in classificatori, quello che adotteremo in Notebook 3
3. **Ensemble:** BalancedRandomForest, EasyEnsemble
4. **Metriche:** F1-score, G-mean invece di Accuracy

---

In [ ]:
#Classe minoritaria:    Infiltration (39 samples)
# STEP 1: Calcolo Statistiche Distribuzione

print("FASE 4: ANALISI DISTRIBUZIONE CLASSI")

# Conta occorrenze per ogni classe
class_counts = df_raw['Label'].value_counts()
total_samples = len(df_raw)

print(" Distribuzione Classi:\n")
class_stats = []
for label, count in class_counts.items():
    percentage = 100 * count / total_samples
    class_stats.append({
        'Label': label,
        'Count': count,
        'Percentage': percentage
    })
    print(f"   {label:<20} : {count:>10,} ({percentage:>5.3f}%)")


# STEP 2: Calcolo Metriche Imbalance

max_class = class_counts.max()
min_class = class_counts.min()
imbalance_ratio = max_class / min_class

print(f"\n⚖️  METRICHE CLASS IMBALANCE:")
print(f"   Classe maggioritaria:  {class_counts.idxmax()} ({max_class:,} samples)")
print(f"   Classe minoritaria:    {class_counts.idxmin()} ({min_class:,} samples)")
print(f"   Imbalance Ratio (IR):  {imbalance_ratio:.1f}:1")
print(f"   Percentuale min class: {100 * min_class / total_samples:.3f}%")


# STEP 3: Visualizzazione Distribuzione (Linear + Log Scale)

print("Generazione visualizzazione distribuzione...\n")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Subplot 1: Scala Lineare
colors = plt.cm.tab10(np.linspace(0, 1, len(class_counts)))
bars1 = ax1.bar(range(len(class_counts)), class_counts.values,
                color=colors, edgecolor='black', linewidth=0.7)
ax1.set_xticks(range(len(class_counts)))
ax1.set_xticklabels(class_counts.index, rotation=45, ha='right')
ax1.set_ylabel('Numero di Campioni')
ax1.set_title('Distribuzione Classi (Scala Lineare)', fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Aggiungi valori sopra le barre
for i, (bar, val) in enumerate(zip(bars1, class_counts.values)):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:,}', ha='center', va='bottom', fontsize=9)

# Subplot 2: Scala Logaritmica
bars2 = ax2.bar(range(len(class_counts)), class_counts.values,
                color=colors, edgecolor='black', linewidth=0.7)
ax2.set_yscale('log')
ax2.set_xticks(range(len(class_counts)))
ax2.set_xticklabels(class_counts.index, rotation=45, ha='right')
ax2.set_ylabel('Numero di Campioni (Log Scale)')
ax2.set_title('Distribuzione Classi (Scala Logaritmica)', fontweight='bold')
ax2.grid(True, alpha=0.3, which='both', axis='y')

plt.suptitle('Analisi Distribuzione Attacchi - CIC-IDS-2017',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(IMG_PATH, '01_class_distribution.png'),
            dpi=300, bbox_inches='tight')
plt.show()
plt.close()

print("   Grafico salvato: 01_class_distribution.png")


# STEP 5: Salvataggio Statistiche

class_stats_df = pd.DataFrame(class_stats)
save_report(class_stats_df, '04_class_distribution_stats.csv', 'csv')

print()

### Interpretazione della Distribuzione

**Impatto sul modeling:**
1. **Accuracy è metrica ingannevole:** Un classificatore che predice sempre `Benign` otterrebbe comunque accuracy alta
2. **Classi rare critiche:** `Infiltration` è la minaccia più pericolosa. Con solo 45 samples, alto rischio di overfitting o mancato apprendimento.


**Soluzione:**
> - Cross-validation stratificata obbligatoria
> - F1-score pesato come metrica primaria

**Prossimo step:** Dataset overview e statistiche descrittive feature.

---

## SEZIONE 7: Dataset Overview & Basic Statistics

### Obiettivo

Fornire una **panoramica completa** del dataset pulito:
1. Identificare tipologie di feature (numeriche vs categoriche)
2. Calcolare statistiche descrittive (media, mediana, std, min/max)
3. Verificare integrità finale (no missing, no duplicates)

### Feature CIC-IDS-2017

Il dataset contiene **79 feature** estratte da flussi di rete usando **CICFlowMeter**:

**Categorie feature:**
- **Flow-based (8):** Duration, Packet count, Byte count, Rate
- **Temporal (12):** IAT (Inter-Arrival Time) statistics, Active/Idle times
- **Packet size (14):** Min/Max/Mean/Std per direzione (Forward/Backward)
- **TCP Flags (9):** PSH, URG, FIN, SYN, ACK, RST, ECE, CWR counts
- **Protocol-specific (6):** Subflow metrics, Bulk statistics
- **Advanced Packet Metrics (30):** Header length, segment size, down/up ratio, etc.

### Statistiche Descrittive Chiave

| Statistica | Significato           | Utilità per IDS |
|------------|-----------------------|----------------|
| **Mean** | Valore medio          | Baseline traffico normale |
| **Std** | Variabilità           | Alta std → possibile anomalia |
| **Skewness** | Asimmetria            | > 2 → distribuzione code pesanti |
| **Kurtosis** | Pesantezza delle code | > 3 → outliers significativi |
| **Zeros %** | Percentuale zeri      | Alta % → possibile low-variance |

---

In [ ]:
# STEP 1: Identificazione Tipologie Feature
print("FASE 5: DATASET OVERVIEW")
print()

# Identifica colonne numeriche
numeric_features = df_raw.select_dtypes(include=[np.number]).columns.tolist()

# Rimuovi label encoded dalla lista feature (non è una feature vera)
if 'Label_Encoded' in numeric_features:
    numeric_features.remove('Label_Encoded')

# STEP 2: Statistiche Descrittive Base
print("Statistiche Descrittive (sample prime 5 features):\n")

# Mostra sample di 5 feature per leggibilità
desc_stats = df_raw[numeric_features[:5]].describe()
print(desc_stats)
print()

# Calcola statistiche complete su tutte le feature
print("Calcolo statistiche complete su tutte le 78 feature...")
full_desc = df_raw[numeric_features].describe().T

# Salva in CSV per consultazione dettagliata
full_desc.to_csv(os.path.join(REPORTS_PATH, '05_descriptive_statistics.csv'))
print(f"   Statistiche complete salvate: 05_descriptive_statistics.csv")
print(f"      (contiene mean, std, min, 25%, 50%, 75%, max per ogni feature)")
print()


# STEP 3: Integrity Check Finale

print(" Final Integrity Check:")
print()

# Verifica missing values
missing_total = df_raw.isnull().sum().sum()
print(f"   Missing values (NaN):  {missing_total}")
if missing_total > 0:
    print("        WARNING: Trovati NaN residui!")
else:
    print("      Nessun valore mancante")

# Verifica duplicati
duplicates_total = df_raw.duplicated().sum()
print(f"   Duplicate rows:        {duplicates_total}")
if duplicates_total > 0:
    print("        WARNING: Trovati duplicati residui!")
else:
    print("      Nessun duplicato")

# Verifica infiniti
inf_total = np.isinf(df_raw.select_dtypes(include=[np.number])).sum().sum()
print(f"   Infinite values:       {inf_total}")
if inf_total > 0:
    print("        WARNING: Trovati infiniti residui!")
else:
    print("      Nessun infinito")

# Memory footprint
memory_mb = df_raw.memory_usage(deep=True).sum() / (1024 ** 2)
print(f"   Memory usage:          {memory_mb:.2f} MB")
print()

print("Dataset pronto per EDA")
print()

**Prossimo step:** Zero/near-zero variance detection per identificare feature non informative.

---

## SEZIONE 8: Zero e Near-Zero Variance Detection

### Obiettivo

Identificare feature con **varianza nulla o trascurabile** che non contribuiscono alla discriminazione tra classi.

### Varianza e Information Content

Una feature con varianza zero (tutti valori identici) ha **zero information content** per la classificazione:



---

In [ ]:
# STEP 1: Rilevamento Zero Variance

print("FASE 6: ZERO E NEAR-ZERO VARIANCE DETECTION")
print()

print("Step 1: Rilevamento feature con varianza zero...")

zero_var_features = []

for col in numeric_features:
    # Una feature ha varianza zero se ha un solo valore unico
    if df_raw[col].nunique() == 1:
        zero_var_features.append(col)

if zero_var_features:
    print(f"     Trovate {len(zero_var_features)} feature a varianza zero:\n")
    for feat in zero_var_features:
        unique_val = df_raw[feat].iloc[0]
        print(f"      - {feat:<50} (valore costante: {unique_val})")
else:
    print("   Nessuna feature a varianza zero")

print()

# STEP 2: Rilevamento Near-Zero Variance (<1% unique values)

print("Step 2: Rilevamento near-zero variance (<1% valori unici)...")
print()

# Usa VarianceThreshold di scikit-learn per approccio robusto
# Threshold 0.01 = 1% varianza normalizzata
selector = VarianceThreshold(threshold=0.01)

# Fit su sample stratificato per performance
sample_size = min(50000, len(df_raw))
print(f"   Analisi su sample stratificato di {sample_size:,} righe...")
df_sample = df_raw.sample(n=sample_size, random_state=42)

try:
    # Fit del selector
    selector.fit(df_sample[numeric_features])

    # Identifica feature con bassa varianza
    low_var_mask = ~selector.get_support()
    near_zero_var_features = [feat for feat, is_low in zip(numeric_features, low_var_mask) if is_low]

    if near_zero_var_features:
        print(f"     Trovate {len(near_zero_var_features)} feature near-zero variance:\n")

        # Mostra top 10 con percentuale valori unici
        for feat in near_zero_var_features[:10]:
            unique_pct = df_raw[feat].nunique() / len(df_raw) * 100
            zeros_pct = (df_raw[feat] == 0).sum() / len(df_raw) * 100
            print(f"      - {feat:<45} : {unique_pct:.4f}% unici, {zeros_pct:.2f}% zeri")

        if len(near_zero_var_features) > 10:
            print(f"\n      ... e altre {len(near_zero_var_features) - 10} feature")
    else:
        print("   Nessuna feature near-zero variance rilevata")

except Exception as e:
    print(f"     Errore nel calcolo VarianceThreshold: {e}")
    near_zero_var_features = []

print()

# STEP 3: Summary e Salvataggio

# Combina liste (rimuovi duplicati)
all_low_var = list(set(zero_var_features + near_zero_var_features))

print(f"SUMMARY VARIANCE ANALYSIS:")
print(f"   Zero variance:       {len(zero_var_features)} feature")
print(f"   Near-zero variance:  {len(near_zero_var_features)} feature")
print(f"   {'─' * 50}")
print(f"   Totale da rimuovere: {len(all_low_var)} feature ({100 * len(all_low_var) / len(numeric_features):.1f}%)")
print()

# Salva lista per Notebook 3 (Feature Selection)
if all_low_var:
    save_report(all_low_var, '06_low_variance_features.json', 'json')

else:
    print("Nessuna feature low-variance da rimuovere")

print()

## SEZIONE 9: Univariate Analysis (Distribuzioni)

### Obiettivo

Analizzare la **distribuzione univariata** delle feature più informative per:
1. Identificare **skewness** (asimmetria) e **kurtosis** (code pesanti)
2. Rilevare **outliers** attraverso statistiche quartili

---

In [ ]:
# STEP 1: Selezione Top 20 Feature per Varianza


print("FASE 7: UNIVARIATE ANALYSIS (Distribuzioni)")
print()

print("Selezione top 20 feature più informative per varianza...\n")

# Calcola varianza per ogni feature
feature_variances = df_raw[numeric_features].var().sort_values(ascending=False)

# Seleziona top 20
top_20_features = feature_variances.head(20).index.tolist()

print(" TOP 20 FEATURES BY VARIANCE:\n")
for i, feat in enumerate(top_20_features, 1):
    var_val = feature_variances[feat]
    print(f"   {i:2d}. {feat:<50} (Var: {var_val:.2e})")

print()


# STEP 2: Calcolo Statistiche Distribuzione
print("Calcolo statistiche distribuzione per top 20 feature...\n")

univariate_stats = []

for feat in top_20_features:
    data = df_raw[feat]

    # Calcola tutte le statistiche
    stats_dict = {
        'Feature': feat,
        'Mean': data.mean(),
        'Median': data.median(),
        'Std': data.std(),
        'Min': data.min(),
        'Max': data.max(),
        'Skewness': data.skew(),
        'Kurtosis': data.kurtosis(),
        'Q1': data.quantile(0.25),
        'Q3': data.quantile(0.75),
        'IQR': data.quantile(0.75) - data.quantile(0.25),
        'Zeros_%': (data == 0).sum() / len(data) * 100
    }

    univariate_stats.append(stats_dict)

# Crea DataFrame
univariate_df = pd.DataFrame(univariate_stats)

# Mostra sample (prime 10)
print(" Statistiche Univariate (Top 10 features):\n")
display_cols = ['Feature', 'Mean', 'Std', 'Skewness', 'Zeros_%']
print(univariate_df.head(10)[display_cols].to_string(index=False))
print()

# Salva completo
save_report(univariate_df, '07_univariate_statistics.csv', 'csv')


# STEP 3: Visualizzazione Histogram Grid (Top 12 Features)
print("Generazione histogram grid per visualizzazione distribuzioni...\n")

fig, axes = plt.subplots(3, 4, figsize=(18, 12))
axes = axes.flatten()

for idx, feat in enumerate(top_20_features[:12]):
    ax = axes[idx]

    # Estrai dati (rimuovi inf/nan residui per sicurezza)
    data = df_raw[feat].replace([np.inf, -np.inf], np.nan).dropna()

    # Determina se applicare log transform per visualizzazione
    data_range = data.max() - data.min()
    if data.max() > 0 and (data.max() / (data.min() + 1e-9) > 1000):
        # Range molto ampio → usa log scale
        data_plot = np.log10(data + 1)  # +1 per gestire zeri
        xlabel = f'log10({feat})'
    else:
        data_plot = data
        xlabel = feat

    # Plot histogram
    ax.hist(data_plot, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
    ax.set_xlabel(xlabel, fontsize=9)
    ax.set_ylabel('Frequenza', fontsize=9)
    ax.set_title(f'{feat}', fontsize=10, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')

    # Aggiungi linee per mean e median
    mean_val = data.mean()
    median_val = data.median()

    if 'log' in xlabel:
        mean_plot = np.log10(mean_val + 1)
        median_plot = np.log10(median_val + 1)
    else:
        mean_plot = mean_val
        median_plot = median_val

    ax.axvline(mean_plot, color='red', linestyle='--', linewidth=1.5,
               label=f'Mean: {mean_val:.2f}')
    ax.axvline(median_plot, color='green', linestyle='--', linewidth=1.5,
               label=f'Median: {median_val:.2f}')
    ax.legend(fontsize=7, loc='upper right')

plt.suptitle('Distribuzione Univariata - Top 12 Features (by Variance)',
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig(os.path.join(IMG_PATH, '02_univariate_distributions.png'),
            dpi=300, bbox_inches='tight')
plt.show()
plt.close()

print("  Histogram grid salvato: 02_univariate_distributions.png")
print()

## SEZIONE 10: Bivariate Analysis (Feature-Target Correlation)

### Obiettivo

Quantificare la **relazione tra ogni feature e il target** (Benign vs Attack) per:
1. Identificare **feature discriminative** (alta correlazione)
2. Prioritizzare feature per feature selection

### Conversione Target Binario

Per calcolare correlazione, convertiamo il target multi-classe (8 categorie) in **binario**:
- `Benign` → 0
- `Qualsiasi attacco` → 1

Questo permette di identificare feature che discriminano genericamente "anomalie" dal traffico normale.

---

In [ ]:
# STEP 1: Preparazione Target Binario

print(" FASE 8: BIVARIATE ANALYSIS (Feature-Target Correlation)")
print()\

print(" Conversione target multi-classe → binario (Benign vs Attack)...\n")

# Crea target binario: Benign=0, Any Attack=1
y_binary = (df_raw['Label'] != 'Benign').astype(int)

print(f"   Distribuzione target binario:")
print(f"      Benign (0): {(y_binary == 0).sum():>10,} samples ({100*(y_binary==0).sum()/len(y_binary):.2f}%)")
print(f"      Attack (1): {(y_binary == 1).sum():>10,} samples ({100*(y_binary==1).sum()/len(y_binary):.2f}%)")
print()


# STEP 2: Sample Stratificato per Performance

print("Creazione sample stratificato per calcolo correlazioni...\n")

# Sample size: 100k righe (balance tra accuracy e performance)
sample_size = min(100000, len(df_raw))

# Sample casuale (non stratificato perché target binario)
np.random.seed(42)
indices = np.random.choice(len(df_raw), size=sample_size, replace=False)
df_sample = df_raw.iloc[indices]
y_sample = y_binary.iloc[indices]

print(f"   Sample size: {len(df_sample):,} righe")
print(f"   Proporzioni mantenute: Benign {100*(y_sample==0).sum()/len(y_sample):.1f}%, Attack {100*(y_sample==1).sum()/len(y_sample):.1f}%")
print()



# STEP 3: Calcolo Spearman Correlation per Ogni Feature
print("Calcolo Spearman correlation (robusto a outliers e non-linearità)...\n")

target_corr_list = []

for col in numeric_features:
    try:
        # Calcola correlazione Spearman
        corr, p_value = spearmanr(df_sample[col], y_sample, nan_policy='omit')

        target_corr_list.append({
            'Feature': col,
            'Spearman_Corr': abs(corr),  # Valore assoluto per ranking
            'Corr_Sign': np.sign(corr),  # Conserva segno per interpretazione
            'p_value': p_value
        })
    except Exception as e:
        print(f"     Errore su feature {col}: {e}")

# Crea DataFrame e ordina per correlazione assoluta decrescente
corr_df = pd.DataFrame(target_corr_list).sort_values('Spearman_Corr', ascending=False)

print(f" Correlazioni calcolate per {len(corr_df)} feature\n")



# STEP 4: Display Top 20 Feature Correlate

print(" TOP 20 FEATURES CORRELATE CON TARGET (Attack vs Benign):\n")
print(corr_df.head(20)[['Feature', 'Spearman_Corr', 'p_value']].to_string(index=False))
print()

# Salva risultati completi
save_report(corr_df, '08_feature_target_correlation.csv', 'csv')


# STEP 5: Visualizzazione Bar Chart Top 20

print("Generazione bar chart per visualizzazione correlazioni...\n")

plt.figure(figsize=(10, 10))
top_20_corr = corr_df.head(20)

# Colori: rosso per correlazione negativa, blu per positiva
colors = ['red' if sign < 0 else 'steelblue' for sign in top_20_corr['Corr_Sign']]

plt.barh(top_20_corr['Feature'], top_20_corr['Spearman_Corr'],
         color=colors, edgecolor='black', linewidth=0.7)
plt.xlabel('|Spearman Correlation|', fontsize=12, fontweight='bold')
plt.title('Top 20 Features Correlate con Classificazione Attack',
          fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()  # Feature più correlata in alto
plt.grid(axis='x', alpha=0.3)

# Aggiungi legenda colori
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='steelblue', edgecolor='black', label='Correlazione Positiva'),
    Patch(facecolor='red', edgecolor='black', label='Correlazione Negativa')
]
plt.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.savefig(os.path.join(IMG_PATH, '03_feature_target_correlation.png'),
            dpi=300, bbox_inches='tight')
plt.show()
plt.close()

print("   Bar chart salvato: 08_feature_target_correlation.png")
print()

### Interpretazione della Feature-Target Correlation

**Correlazioni negative (ρ < 0, colore rosso):**
- Indicano feature che **diminuiscono** durante attacchi
- Es: `Init Win Bytes` basso in SYN flood (incomplete handshake)


**Prossimo step:** Visualizzare class separability per le top 4 feature.

---

## SEZIONE 11: Class Separability Visualization

### Obiettivo

Visualizzare graficamente quanto le **top feature** separano le **8 classi di attacco**, usando violin plots per mostrare l'intera distribuzione per classe.


**Vantaggi Violin Plot :**
- Mostra **bimodalità** (es. PortScan ha 2 picchi: scansioni lente vs veloci)
- Evidenzia **forma distribuzione** (simmetrica vs skewed)
- Visualizza **overlap tra classi** (se violin si sovrappongono → feature poco discriminativa)

### Interpretazione Separability

**Alta separability:**
- Violin plots **non si sovrappongono**
- Mediane delle classi **distanti**

**Bassa separability:**
- Violin plots **ampiamente sovrapposti**
- Distribuzioni **simili** tra classi

### Implicazioni per Modeling

Feature con **alta separability visiva** sono ottime candidate per:
- **Decision Trees:** Permettono split netti
- **SVM:** Facilitano costruzione iperpiani di separazione

---

In [ ]:
# STEP 1: Sample Stratificato per Visualizzazione

print(" FASE 9: CLASS SEPARABILITY ANALYSIS")
print()

print("Preparazione sample stratificato per visualizzazione...\n")

# Sample max 5000 righe per classe
max_samples_per_class = 5000

# Sampling stratificato robusto
df_viz = pd.concat([
    group.sample(n=min(len(group), max_samples_per_class), random_state=42)
    for label, group in df_raw.groupby('Label')
], ignore_index=True)

# Sanity check
assert 'Label' in df_viz.columns, "ATTENZIONE: 'Label' column missing!"
assert len(df_viz) > 0, "ATTENZIONE: df_viz is empty!"

print(f"   Sample size totale: {len(df_viz):,} righe")
print(f"   Distribuzione classi nel sample:")
for label, count in df_viz['Label'].value_counts().sort_values(ascending=False).items():
    pct = 100 * count / len(df_viz)
    print(f"      {label:<20} : {count:>6,} righe ({pct:>5.2f}%)")
print()

# STEP 2: Selezione Top 4 Features
print(" Selezione top 4 features da analisi correlazione...\n")

# Prendi top 4 dalla correlation analysis precedente
top_4_features = corr_df.head(4)['Feature'].tolist()

print("   Feature selezionate per separability visualization:")
for i, feat in enumerate(top_4_features, 1):
    corr_val = corr_df[corr_df['Feature'] == feat]['Spearman_Corr'].values[0]
    print(f"      {i}. {feat:<50} (|ρ| = {corr_val:.3f})")
print()


# STEP 3: Generazione Grid 2x2 di Violin Plots
print("Generazione violin plots per separability visualization...\n")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, feat in enumerate(top_4_features):
    ax = axes[idx]

    # Violin plot con quartili interni
    sns.violinplot(
        x='Label',
        y=feat,
        data=df_viz,
        ax=ax,
        palette='Set2',      # Colori distinti per classe
        inner='quartile',    # Mostra Q1, Q2, Q3
        scale='width',       # Larghezza proporzionale a count
        cut=0                # No estensione oltre min/max
    )

    ax.set_title(f'Distribuzione: {feat}', fontsize=12, fontweight='bold')
    ax.set_xlabel('', fontsize=10)
    ax.set_ylabel(feat, fontsize=10)
    ax.tick_params(axis='x', rotation=45, labelsize=9)
    ax.grid(True, alpha=0.3, axis='y')

    # Applica log scale se range molto ampio
    data_range = df_viz[feat].max() / (df_viz[feat].min() + 1e-9)
    if data_range > 1000:
        ax.set_yscale('log')
        ax.set_ylabel(f'{feat} (log scale)', fontsize=10)

plt.suptitle('Class Separability Analysis - Top 4 Features',
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig(os.path.join(IMG_PATH, '04_class_separability.png'),
            dpi=300, bbox_inches='tight')
plt.show()
plt.close()

print("   Violin plots salvati: 04_class_separability.png")
print()

## SEZIONE 12: Multicollinearity Detection

### Obiettivo

Identificare **coppie di feature altamente correlate** (ridondanti) per:
1. Ridurre **dimensionalità** senza perdita di informazione
2. Migliorare **interpretabilità** modelli (evitare feature duplicate)
3. Velocizzare **training** rimuovendo ridondanza

### Teoria: Multicollinearità

Due feature $X_1$ e $X_2$ sono **multicollineari** se:
$$|Corr(X_1, X_2)| > \theta$$

dove $\theta$ è una soglia.

### Strategia di Rimozione

Per ogni coppia $(X_1, X_2)$ con $|r| > 0.95$:
1. **Calcola** correlazione con target per entrambe
2. **Mantieni** la feature più correlata al target
3. **Rimuovi** l'altra

---

In [ ]:
# STEP 1: Calcolo Matrice di Correlazione (Pearson)
print(" FASE 10: MULTICOLLINEARITY DETECTION")
print()

print("Calcolo matrice di correlazione (Pearson) tra feature...\n")

# Sample per performance (50k righe sufficienti per correlazioni stabili)
sample_corr = df_raw[numeric_features].sample(
    n=min(50000, len(df_raw)),
    random_state=42
)

print(f"   Sample size: {len(sample_corr):,} righe")
print(f"   Feature: {len(numeric_features)}")
print(f"   Matrice dimensione: {len(numeric_features)} × {len(numeric_features)}")
print()
print("   Calcolo in corso (può richiedere 1-2 minuti)...")

# Calcola matrice correlazione (usa Pearson per linearità)
corr_matrix = sample_corr.corr(method='pearson').abs()

print("   Matrice calcolata")
print()


# STEP 2: Estrazione Coppie con Alta Correlazione (|r| > 0.95)
print("Ricerca coppie altamente correlate (|r| > 0.95)...\n")

# Estrai upper triangle (evita duplicati e diagonale)
upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

# Trova coppie con correlazione > 0.95
high_corr_pairs = []

for column in upper_triangle.columns:
    # Feature correlate con 'column'
    correlated_features = upper_triangle.index[upper_triangle[column] > 0.95].tolist()

    for corr_feat in correlated_features:
        high_corr_pairs.append({
            'Feature_1': column,
            'Feature_2': corr_feat,
            'Correlation': upper_triangle.loc[corr_feat, column]
        })

if high_corr_pairs:
    pairs_df = pd.DataFrame(high_corr_pairs).sort_values('Correlation', ascending=False)

    print(f"     Trovate {len(pairs_df)} coppie con |r| > 0.95:\n")
    print(pairs_df.head(15).to_string(index=False))

    if len(pairs_df) > 15:
        print(f"\n   ... e altre {len(pairs_df) - 15} coppie")
else:
    print("   Nessuna coppia con correlazione > 0.95 trovata")
    pairs_df = pd.DataFrame()

print()


# STEP 3: Identificazione Candidati per Rimozione

if not pairs_df.empty:
    print(" Identificazione feature da rimuovere...\n")

    # Strategia semplice: mantieni Feature_1, rimuovi Feature_2 in ogni coppia
    # (Feature_1 appare prima alfabeticamente/per importanza)
    drop_candidates = list(set(pairs_df['Feature_2'].tolist()))

    print(f"    Candidate per rimozione: {len(drop_candidates)} feature")
    print(f"      (strategia: mantieni Feature_1 in ogni coppia)\n")

    # Mostra prime 10
    for i, feat in enumerate(drop_candidates[:10], 1):
        print(f"      {i:2d}. {feat}")

    if len(drop_candidates) > 10:
        print(f"      ... e altre {len(drop_candidates) - 10} feature")

    # Salva report
    save_report(pairs_df, '09_multicollinearity_pairs.csv', 'csv')
    # Salva lista drop candidates in JSON leggibile
    if drop_candidates:
        save_report(drop_candidates, '10_multicollinearity_drop_candidates.json', 'json')


    print()
else:
    drop_candidates = []
    print("Nessuna feature ridondante da rimuovere")
    print()


# STEP 4: Visualizzazione Heatmap (Top 20 Features)

if not pairs_df.empty:
    print("Generazione heatmap per visualizzazione correlazioni...\n")

    # Seleziona top 20 feature da feature-target correlation
    top_20_for_heatmap = corr_df.head(20)['Feature'].tolist()

    # Filtra solo feature presenti in corr_matrix
    top_20_in_matrix = [f for f in top_20_for_heatmap if f in corr_matrix.columns]

    plt.figure(figsize=(14, 12))
    sns.heatmap(
        corr_matrix.loc[top_20_in_matrix, top_20_in_matrix],
        annot=False,  # No numeri (troppi per leggibilità)
        cmap='coolwarm',  # Rosso=alta, Blu=bassa
        center=0,
        vmin=-1,
        vmax=1,
        square=True,
        linewidths=0.5,
        cbar_kws={'label': 'Pearson Correlation', 'shrink': 0.8}
    )
    plt.title('Correlation Heatmap - Top 20 Features',
              fontsize=14, fontweight='bold')
    plt.xticks(rotation=45, ha='right', fontsize=8)
    plt.yticks(rotation=0, fontsize=8)
    plt.tight_layout()
    plt.savefig(os.path.join(IMG_PATH, '05_correlation_heatmap.png'),
                dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

    print("   Heatmap salvata: 05_correlation_heatmap.png")
    print()

print("Multicollinearity detection completata")
print()

## SEZIONE 13 (DEPRECATA e SPOSTATA NELLA SEZIONE SUCCESSIVA NOTEBOOK 02): 
Feature Selection Preparation (Information Gain & ANOVA)

### Obiettivo

Applicare **metodi statistici di feature selection** per creare un **consensus ranking** delle feature più informative:
1. **Mutual Information (MI):** Misura dipendenza non-lineare feature-target
2. **ANOVA F-statistic:** Testa differenza medie tra classi
3. **Consensus Ranking:** Combina 3 metriche (MI + ANOVA + Spearman) per ridurre bias

### Consensus Ranking: Best of Both Worlds

Combinando 3 metriche diverse riduciamo **bias algoritmico**:
- Feature ranked alta da **tutte e 3** → robustamente informativa


---

In [ ]:
# STEP 1: Sample Stratificato per Computational Efficiency
print(" FASE 11: FEATURE SELECTION METRICS (Information Gain & ANOVA)")
print()

print("Preparazione sample stratificato per calcoli computazionalmente costosi...\n")

#  METODO ROBUSTO: Sample per gruppo mantenendo struttura
max_samples_per_class = 5000

df_fs_sample = pd.concat([
    group.sample(n=min(len(group), max_samples_per_class), random_state=42)
    for label, group in df_raw.groupby('Label')
], ignore_index=True)

# Sanity check critico
assert 'Label' in df_fs_sample.columns, " ATTENZIONE: 'Label' column missing!"
assert len(df_fs_sample) > 0, " ATTENZIONE: df_fs_sample is empty!"

X_sample = df_fs_sample[numeric_features].fillna(0)  # MI non tollera NaN
y_sample = df_fs_sample['Label']

print(f"    Sample size: {len(df_fs_sample):,} righe")
print(f"   Classi: {len(y_sample.unique())}")
print(f"   Feature: {len(numeric_features)}")
print()

# Mostra distribuzione sample
print("   Distribuzione classi nel sample:")
for label, count in y_sample.value_counts().sort_values(ascending=False).items():
    pct = 100 * count / len(df_fs_sample)
    print(f"      {label:<20} : {count:>5,} righe ({pct:>5.2f}%)")
print()


# STEP 2: Calcolo Mutual Information (Information Gain)
print("Step 1: Calcolo Mutual Information...\n")
print("   Questo può richiedere 2-3 minuti...")

try:
    # Calcola MI per classificazione multi-classe
    mi_scores = mutual_info_classif(
        X_sample,
        y_sample,
        discrete_features=False,  # Feature continue
        n_neighbors=5,            # Parametro KNN per stima densità
        random_state=42
    )

    # Crea DataFrame risultati
    mi_df = pd.DataFrame({
        'Feature': numeric_features,
        'MI_Score': mi_scores
    }).sort_values('MI_Score', ascending=False)

    print("\n    Mutual Information calcolata\n")
    print(" TOP 20 FEATURES BY MUTUAL INFORMATION:\n")
    print(mi_df.head(20).to_string(index=False))
    print()

    # Salva risultati
    save_report(mi_df, '11_mutual_information_scores.csv', 'csv')

except Exception as e:
    print(f"\n     Errore nel calcolo MI: {e}")
    print("   Creazione placeholder con score=0...\n")
    mi_df = pd.DataFrame({
        'Feature': numeric_features,
        'MI_Score': 0
    })


# STEP 2b: Visualizzazione Mutual Information

if mi_df['MI_Score'].sum() > 0:  # Solo se calcolo ha successo
    print("Generazione bar chart Mutual Information...\n")

    plt.figure(figsize=(10, 10))
    top_20_mi = mi_df.head(20)

    plt.barh(top_20_mi['Feature'], top_20_mi['MI_Score'],
             color='coral', edgecolor='black', linewidth=0.7)
    plt.xlabel('Mutual Information Score', fontsize=12, fontweight='bold')
    plt.title('Top 20 Features by Information Gain',
              fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(IMG_PATH, '06_mutual_information.png'),
                dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

    print("   Bar chart salvato: 06_mutual_information.png")
    print()


# STEP 3: Calcolo ANOVA F-Statistics

print("Step 2: Calcolo ANOVA F-Statistics...\n")

try:
    # Calcola F-statistic e p-value per ogni feature
    F_stats, p_values = f_classif(X_sample, y_sample)

    # Crea DataFrame risultati
    anova_df = pd.DataFrame({
        'Feature': numeric_features,
        'F_Statistic': F_stats,
        'p_value': p_values
    }).sort_values('F_Statistic', ascending=False)

    print("   ANOVA F-Statistics calcolata\n")
    print(" TOP 20 FEATURES BY ANOVA F-STATISTIC:\n")
    print(anova_df.head(20)[['Feature', 'F_Statistic', 'p_value']].to_string(index=False))
    print()

    # Salva risultati
    save_report(anova_df, '12_anova_f_statistics.csv', 'csv')

except Exception as e:
    print(f"     Errore nel calcolo ANOVA: {e}")
    print("   Creazione placeholder con F=0...\n")
    anova_df = pd.DataFrame({
        'Feature': numeric_features,
        'F_Statistic': 0,
        'p_value': 1
    })


# STEP 4: Creazione Consensus Ranking

print("  Step 3: Creazione Consensus Ranking (MI + ANOVA + Correlation)...\n")

try:
    # Inizializza scaler per normalizzazione
    scaler = MinMaxScaler()

    # Crea DataFrame base
    consensus_df = pd.DataFrame({'Feature': numeric_features})

    # Merge tutti gli score
    consensus_df = consensus_df.merge(
        mi_df[['Feature', 'MI_Score']], on='Feature', how='left'
    ).merge(
        anova_df[['Feature', 'F_Statistic']], on='Feature', how='left'
    ).merge(
        corr_df[['Feature', 'Spearman_Corr']], on='Feature', how='left'
    )

    # Riempi eventuali NaN con 0
    consensus_df.fillna(0, inplace=True)

    # Normalizza ciascuno score a [0, 1]
    consensus_df['MI_Norm'] = scaler.fit_transform(consensus_df[['MI_Score']])
    consensus_df['ANOVA_Norm'] = scaler.fit_transform(consensus_df[['F_Statistic']])
    consensus_df['Corr_Norm'] = consensus_df['Spearman_Corr']  # Già in [0,1]

    # Calcola Consensus Score (media semplice dei 3 score normalizzati)
    consensus_df['Consensus_Score'] = (
        consensus_df['MI_Norm'] +
        consensus_df['ANOVA_Norm'] +
        consensus_df['Corr_Norm']
    ) / 3

    # Ordina per Consensus Score decrescente
    consensus_df = consensus_df.sort_values('Consensus_Score', ascending=False)

    print("   Consensus Ranking creato\n")
    print(" TOP 25 FEATURES BY CONSENSUS RANKING:\n")
    display_cols = ['Feature', 'Consensus_Score', 'MI_Score', 'F_Statistic', 'Spearman_Corr']
    print(consensus_df.head(25)[display_cols].to_string(index=False))
    print()

    # Salva risultato completo
    save_report(consensus_df, '13_consensus_feature_ranking.csv', 'csv')

except Exception as e:
    print(f"     Errore nella creazione consensus: {e}")
    print("   Utilizzare ranking individuali (MI, ANOVA, o Correlation)")

print()

## SEZIONE 14: Domain-Specific Analysis (Network Features) -> NESSUN RISULTATO UTILE - DEPRECATA

### Obiettivo

Analizzare **feature specifiche del dominio networking** che hanno significato semantico per Intrusion Detection:
1. **TCP Flags:** PSH, URG, FIN, SYN, ACK, RST, ECE, CWR
2. **Protocol features:** Protocollo trasporto, flow direction
3. **Attack signatures:** Pattern noti in letteratura security

### TCP Flags: Semantica di Rete

I **TCP flags** sono bit nel header TCP che controllano lo stato della connessione:


---

In [ ]:
# STEP 1: Identificazione Feature Domain-Specific

print(" FASE 12: DOMAIN-SPECIFIC ANALYSIS (Network Features)")
print()

# Identifica feature TCP flags (contengono 'Flag' o 'flag' nel nome)
tcp_flag_features = [col for col in df_raw.columns if 'Flag' in col or 'flag' in col.lower()]

# Identifica feature Protocol
protocol_features = [col for col in df_raw.columns if 'Protocol' in col]

print(f" Feature TCP Flags identificate: {len(tcp_flag_features)}\n")
for feat in tcp_flag_features:
    print(f"   - {feat}")

if protocol_features:
    print(f"\n Feature Protocol identificate: {len(protocol_features)}\n")
    for feat in protocol_features:
        print(f"   - {feat}")

print()


# STEP 2: Analisi TCP Flags per Attack Type

if tcp_flag_features:
    print("Analisi TCP Flags per Attack Type:\n")

    # Calcola media per ogni flag per ogni classe
    flag_analysis = df_raw.groupby('Label')[tcp_flag_features].mean()

    print("   Media conteggio TCP Flags per classe:\n")
    print(flag_analysis.to_string())
    print()

    # Identifica pattern anomali
    print("\nPattern Anomali Identificati:\n")

    for label in flag_analysis.index:
        row = flag_analysis.loc[label]

        # Pattern specifici per attack type
        if label == 'PortScan':
            if 'FIN Flag Count' in row.index and row['FIN Flag Count'] > 0.5:
                print(f"     {label}: Alto FIN count → Possibile FIN scan")
            if 'SYN Flag Count' in row.index and row['SYN Flag Count'] > row.get('ACK Flag Count', 0) * 2:
                print(f"     {label}: SYN >> ACK → SYN scan (incomplete handshake)")

        elif label in ['DoS', 'DDoS']:
            if 'SYN Flag Count' in row.index and row['SYN Flag Count'] > 1.0:
                print(f"     {label}: SYN elevato → Possibile SYN flood")
            if 'RST Flag Count' in row.index and row['RST Flag Count'] > 0.5:
                print(f"     {label}: RST elevato → Connection disruption")

        elif label == 'WebAttack':
            if 'PSH Flag Count' in row.index and row['PSH Flag Count'] > 0.8:
                print(f"     {label}: PSH elevato → HTTP payload attacks (SQLi, XSS)")

    if not any(['PortScan' in flag_analysis.index, 'DoS' in flag_analysis.index]):
        print("   Pattern TCP flags coerenti con traffico normale")

    print()


# STEP 3: Visualizzazione Heatmap TCP Flags

if tcp_flag_features and len(tcp_flag_features) > 0:
    print("Generazione heatmap TCP Flags per Attack Type...\n")

    plt.figure(figsize=(12, 6))
    sns.heatmap(
        flag_analysis.T,  # Transpose: flags su righe, classi su colonne
        annot=True,  # Mostra valori numerici
        fmt='.2f',  # 2 decimali
        cmap='YlOrRd',  # Giallo → Arancione → Rosso
        cbar_kws={'label': 'Mean Flag Count'},
        linewidths=0.5,
        linecolor='gray'
    )
    plt.title('TCP Flags Average by Attack Type',
              fontsize=14, fontweight='bold')
    plt.xlabel('Attack Type', fontsize=12)
    plt.ylabel('TCP Flag', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(IMG_PATH, '07_tcp_flags_analysis.png'),
                dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

    print("   Heatmap salvata: 07_tcp_flags_analysis.png")
    print()
else:
    print("    Nessuna feature TCP flag disponibile per visualizzazione")
    print()


# STEP 4: Analisi Protocol Distribution (se presente)

if protocol_features:
    print("Analisi distribuzione Protocol per Attack Type:\n")

    for feat in protocol_features:
        print(f"   Feature: {feat}\n")

        # Distribuzione protocol per classe
        protocol_dist = df_raw.groupby('Label')[feat].value_counts(normalize=True).unstack(fill_value=0)

        print(protocol_dist.to_string())
        print()

        # Identifica protocol dominanti per attack
        print("   Protocol Dominanti per Attack:\n")
        for label in protocol_dist.index:
            top_protocol = protocol_dist.loc[label].idxmax()
            top_pct = protocol_dist.loc[label].max() * 100
            print(f"      {label:<20} : {top_protocol} ({top_pct:.1f}%)")
        print()
else:
    print("Nessuna feature Protocol disponibile nel dataset")
    print()

print("Domain-specific analysis completata")
print()

In [ ]:
# ═══════════════════════════════════════════════════════════════
# SEZIONE FINALE: SALVATAGGIO DATASET PULITO
# ═══════════════════════════════════════════════════════════════

print("="*80)
print("FASE 16 - SALVATAGGIO DATASET PULITO PER FEATURE ENGINEERING")
print("="*80)
print()

# Prepara dataset per export
df_export = df_raw.copy()

# SALVA PARQUET (più veloce, consigliato)
parquet_path = os.path.join(OUTPUT_PATH, "cicids2017_cleaned.parquet")
print(f"Salvataggio Parquet: {parquet_path}")
df_export.to_parquet(parquet_path, index=False, compression='snappy')
parquet_size = os.path.getsize(parquet_path) / (1024**2)

# SALVA CSV (backup)
csv_path = os.path.join(OUTPUT_PATH, "cicids2017_cleaned.csv")
print(f"Salvataggio CSV: {csv_path}")
df_export.to_csv(csv_path, index=False)
csv_size = os.path.getsize(csv_path) / (1024**2)

print()
print(" Dataset cleaned salvato con successo!")
print(f"   Righe:           {df_export.shape[0]:,}")
print(f"   Colonne:         {df_export.shape[1]}")
print(f"   Size Parquet:    {parquet_size:.1f} MB")
print(f"   Size CSV:        {csv_size:.1f} MB")
print()
print("Prossimo step: Eseguire FEATURE ENGINEERING per feature selection")
print("="*80)
